# Assignment: Economic Connectedness

>Ioanna Moraiti <br />
>AM: 8190121 <br />
>Email: t8190121@aueb.gr

Η εργασία βασίζεται στα δύο paper του περιοδικού Nature:
* https://doi.org/10.1038/s41586-022-04996-4
* https://doi.org/10.1038/s41586-022-04997-3

Για την δημιουργία των παρακάτω διαγραμμάτων χρησιμοποίησα τα module της python pandas, matplotlib, numpy, plotly, 
geopandas, seaborn και plotnine. Επομένως είναι απαραίτητη η εγκατάσταση των εκάστοτε πακέτων.
Τα δεδομένα που χρησιμοποιήθηκαν προέρχονται από τον [Social Capital Atlas Datasets](https://data.humdata.org/dataset/social-capital-atlas) και το [Replication Package](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NKCQM1) του [Opportunity Atlas](https://www.nber.org/papers/w25147).

## Question 1

Το πρώτο ερώτημα αφορούσε την αντιγραφή του [2ου Διαγράμματος του 1ου paper](https://www.nature.com/articles/s41586-022-04996-4/figures/2). Ξεκινώντας κάνω τα απαραίτητα imports.

In [ ]:
import pandas as pd
import plotly.express as px
from urllib.request import urlopen
import numpy as np
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

Το json περιέχει γεωγραφικά στοιχεία των ΗΠΑ τα οποία θα χρησιμοποιηθού από την βιβλιοθήκη express της plotly για την δημιουργία του χάρτη. 

Διαβάζω τα απαραίτητα datasets. Αποθηκεύω τον κωδικό των κομμητιών ως string προκειμένου στη συνέχεια να μπορώ να επεξεργαστώ πλήρως την μορφή του. Στον δείκτη economic connectedness των κομμητιών χρησιμοποιώ μία συνάρτηση στρογγυλοποίησης ώστε να μπορώ να πετύχω το ίδιο αποτέλεσμα που ζητείται στο ερώτημα.

In [ ]:
sc_counties = pd.read_csv("social_capital_county.csv", dtype={'county': str})
sc_counties['ec_county'] = sc_counties['ec_county'].apply(lambda x: round(x, 2))

Αφού έχω διαβάσει τα δεδομένα μου, επεξεργάζομαι την μορφή των κωδικών των κωμμητιών ώστε να είναι προσαρμοσμένοι με τα πρότυπα 
[FIPS](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt).

In [ ]:
sc_counties['county'] = sc_counties['county'].apply(lambda x: '0' + x if len(x) == 4 else x)
fips = sc_counties['county']

Για την εμφάνιση διακριτών χρωμάτων στον χάρτι βάζω τα δεδομένα μου σε bins, σε μορφή όπως αναγράφονται στον πίνακα labels. Για να συμπεριληφθούν και οι τιμές NaN, τους αναθέτω μία άκυρη τιμή. Συγκεκριμένα εδώ τους αναθέτω την τιμή -9 και προσθέτων ένα διάστημα (-10, 0] για όλες τις NaN τιμές. Στη συνέχεια, για την σωστή αναπαράστασή τους στο διάγραμμα τους αναθέτω ως τιμή το string ΝΑ.

In [ ]:
values = pd.DataFrame({'ec': sc_counties['ec_county']})
bins =  [-10, 0, 0.58, 0.67, 0.72, 0.76, 0.81, 0.85, 0.9, 0.97, 1.06, 100]
labels= ['NA', '<0.58', '0.58-0.67', '0.67-0.72', '0.72-0.76', '0.76-0.81', 
         '0.81-0.85', '0.85-0.90', '0.90-0.97', '0.97-1.06', '>1.06']
fip_values = pd.DataFrame({'fips': fips, 
                           'ec':values['ec'], 
                           'name': sc_counties['county_name']})
fip_values['ec'].fillna(value=-9, inplace=True)
fip_values['bins'] = pd.cut(fip_values['ec'],
                            bins=bins,
                            labels=labels,
                            include_lowest=False,
                            right=True)
fip_values['ec'] = fip_values['ec'].apply(lambda x: 'NA' if x == -9 else x)

Αφού έχω τα δεδομένα μου στην μορφή που χρειάζεται για τη δημιουργία του διαγράμματος, ξεκινάω να φτιάχνω το διάγραμμα. Αρχικά θα φτιάξω έναν πίνακα με τους hex κωδικούς των χρωμάτων που θα χρησιμοποιήσω

In [ ]:
colorscale = ['#ffd700', '#b2182b', '#d6604d', '#f4a582', '#fddbc7', '#f7f7f7', '#d1e5f0', '#92c5de', '#4393c3', '#2166ac', '#053061']

Για τη δημιουργία του χάρτι χρησιμοποιώ τη βιβλιοθήση express της plotly και συγκεκριμένα την συνάρτηση [express.choropleth_mapbox](https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth_mapbox.html). Χρησιμοποιείται, όπως αναφέρθηκε και πιο πριν, το json αρχείο με τα γεωγραφικά στοιχεία. Το χρώμα των χωρών σχετίζεται με το σχετικό bin της economic connectedness.

In [ ]:
fig = px.choropleth_mapbox(fip_values,
                           geojson=counties,
                           locations='fips',
                           color='bins',
                           mapbox_style='white-bg',
                           zoom=3,
                           center={"lat": 37.0902, "lon": -95.7129},
                           labels={'ec': 'economic connectedness', 
                                   'fips': 'county'},
                           hover_name='name',
                           hover_data={'fips': True, 
                                       'ec': True, 
                                       'bins': False},
                           color_discrete_sequence=colorscale,
                           category_orders={'bins': labels}
                           )

Τέλος, κάνω κάποιες μικρές αλλαγές στο legend του διαγράμματος και το εμφανίζω.

In [ ]:
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, legend={'title': 'Economic Connectedness'}, legend_traceorder="reversed")
fig.show()

## Question 2

Στο δεύτερο ερώτημα απαιτείται η αντιγραφή του [4ου διαγράμματος του 1ου paper](nature.com/articles/s41586-022-04996-4/figures/4). Για το συγκεκριμένο διάγραμμα θα χρησιμοποιήσω την βιβλιοθήκη seaborn.

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

Στον δείκτη upward mobility αντιστοιχεί η μεταβλητή kfr_pooled_pooled_p25, που δείχνει για τα άτομα όλων των εθνικοτήτων που ανήκαν στο 25ο percentile την τωρινή εισοδηματική τους κατάταξη. Πριν ξεκινήσω την επεξεργασία των δεδομένων, φέρνω τους κωδικού των κομμητιών στην κατάλληλη μορφή.

In [ ]:
pooled_df = pd.read_csv('county_outcomes_simple.csv',
                        usecols=['state', 'county', 'kfr_pooled_pooled_p25'],
                        dtype={'state': str, 
                               'county': str, 
                               'kfr_pooled_pooled_p25': float, 
                               'kfr_pooled_pooled_p25_se': float})
pooled_df['state'] = pooled_df['state'].apply(lambda x: '0' + x if len(x) == 1 else x)
pooled_df['county'] = pooled_df['county'].apply(lambda x: '0' + x if len(x) == 2 else ('00' + x if len(x) == 1 else x))
pooled_df['fips'] = pooled_df['state'] + pooled_df['county']
pooled_df.drop(columns=['state', 'county'], inplace=True)

Το ίδιο κάνω και στα δεδομένα που περιέχουν τον δείκτη economic connectedness.

In [ ]:
sc_counties = pd.read_csv("social_capital_county.csv", 
                          dtype={'county': str},
                          usecols=['county', 
                                   'county_name', 
                                   'ec_county'])
sc_counties['fips'] = sc_counties['county'].apply(lambda x: '0' + x if len(x) == 4 else x)
sc_counties.drop(columns=['county'], inplace=True)

Για την εύρεση του πληθυσμού χρησιμοποιώ ένα dataset με δημογραφικά στοιχεία των ΗΠΑ που μπορεί να βρεθεί [εδώ](https://simplemaps.com/data/us-counties).

In [ ]:
pop = pd.read_csv('uscounties.csv', 
                  usecols=['population', 
                           'county_fips'], 
                  dtype={'population': int, 
                         'county_fips': str})
pop.rename(columns={'county_fips': 'fips'}, inplace=True)

Στη συνέχεια κάνω merge τα dataframes με τον πληθυσμό και τον δείκτη economic connectedness, επιλέγω τις 200 πληθυσμιακά μεγαλύτερες κομμητίες και τα ενώνω όλα αυτά στo dataframe income_ec, μαζί με τον δείκτη upward mobility.

In [ ]:
sc_counties = pd.merge(left=sc_counties, right=pop, on='fips')
sc_counties = sc_counties.sort_values(by='population', ascending=False).head(200)
income_ec = pd.merge(left=sc_counties, right=pooled_df, on='fips')

Για τη σωστή απεικόνιση των ονομάτων των πολιτειών και κομμητιών κάνω έναν διαχωρισμό του ονόματός τους.

In [ ]:
income_ec['county'] = income_ec['county_name'].str.split(pat=',', expand=True)[0]

Ξεκινώντας την ζωγραφική των διαγραμμάτων, αρχικά ορίζω τα ασιθητικά στοιχεία. 

Δημιουργώ το βασικό μου διάγραμμα το οποίο είναι ένα lmplot με άξονα $x$ τον δείκτη economic connectedness και άξονα $y$ τον δείκτη upward mobility για τα άτομα που άνοικαν στο 25ο percentile. 

Στο εν λόγο διάγραμμα πρέπει να δείξουμε και τα σημεία που αντιστοιχούν στις πόλεις San Francisco, Salt Lake City, New York, Indianapolis και Minneapolis. Για να δείξουμε αυτές τις πόλεις δείχνουμε τις κομμητίες στις οποίες ανοίκουν και με την συνάρτηση annotate της plotly προσθέτουμε ταμπέλες και βελάκια στα αντίστοιχα σημεία.

In [ ]:
#Aesthetics
sb.set(style="darkgrid")
sb.set_palette(palette='tab10', n_colors=1, color_codes='b')

#Main graph
_ = sb.lmplot(x='ec_county',
              y='kfr_pooled_pooled_p25',
              data=income_ec,
              scatter=True,
              fit_reg=True)
_.set_axis_labels('Economic Connectedness', 
                  'Predicted Household Income Rank\n For Children with Parents at 25th Income Percentile')
_.fig.set_figwidth(8)
_.fig.set_figheight(3)

#Annotate cities
san_francisco = income_ec[income_ec['county'] == 'San Francisco']
plt.annotate(text='San Francisco',
             xy=(san_francisco['ec_county'], san_francisco['kfr_pooled_pooled_p25']),
             xytext=(san_francisco['ec_county'] - 0.14, san_francisco['kfr_pooled_pooled_p25'] - 0.07),
             arrowprops=dict(shrink=0.07, facecolor='black', edgecolor='white', width=2))

salt_lake_city = income_ec[(income_ec['county'] == 'Salt Lake')]
plt.annotate(text='Salt Lake City',
             xy=(salt_lake_city['ec_county'], salt_lake_city['kfr_pooled_pooled_p25']),
             xytext=(salt_lake_city['ec_county'] - 0.04, salt_lake_city['kfr_pooled_pooled_p25'] + 0.05),
             arrowprops=dict(shrink=0.07, facecolor='black', edgecolor='white', width=2))

new_york_city = income_ec[(income_ec['county'] == 'New York')]
plt.annotate(text='New York City',
             xy=(new_york_city['ec_county'], new_york_city['kfr_pooled_pooled_p25']),
             xytext=(new_york_city['ec_county'] - 0.25, new_york_city['kfr_pooled_pooled_p25'] + 0.06),
             arrowprops=dict(shrink=0.07, facecolor='black', edgecolor='white', width=2))

indianapolis = income_ec[(income_ec['county_name'] == 'Marion, Indiana')]
plt.annotate(text='Indianapolis',
             xy=(indianapolis['ec_county'], indianapolis['kfr_pooled_pooled_p25']),
             xytext=(indianapolis['ec_county'] + 0.15, indianapolis['kfr_pooled_pooled_p25'] - 0.01),
             arrowprops=dict(shrink=0.07, facecolor='black', edgecolor='white', width=2))

minneapolis = income_ec[(income_ec['county_name'] == 'Hennepin, Minnesota')]
plt.annotate(text='Minneapolis',
             xy=(minneapolis['ec_county'], minneapolis['kfr_pooled_pooled_p25']),
             xytext=(minneapolis['ec_county'] + 0.05, minneapolis['kfr_pooled_pooled_p25'] - 0.05),
             arrowprops=dict(shrink=0.07, facecolor='black', edgecolor='white', width=2))
plt.show()

## Question 3

Για το τρίτο ερώτημα θα φτίαξω το [6ο Διάγραμμα του 1ου paper](https://www.nature.com/articles/s41586-022-04996-4/figures/6). Ξεκινάω διαβάζοντας τα απαραίτητα αρχεία csv. Το αρχείο zip_ec περιέχει δείκτες economic connectedness για κάθε ZIP code των ΗΠΑ, ενώ το αρχείο zip_income περιέχει το μέσο εισόδημα ανά ZIP code και το upward economic mobility για άτομα που ανοίκουν στο 25ο percentile της εισοδηματικής κατάταξης.

In [ ]:
zip_ec = pd.read_csv('social_capital_zip.csv', 
                     usecols=['zip', 'ec_zip'])
zip_income = pd.read_stata('zip_covariates.dta', 
                           columns=['zip', 
                                    'kfr_pooled_pooled_p25', 
                                    'med_inc_2018'])
zip_income['kfr_pooled_pooled_p25'] = zip_income['kfr_pooled_pooled_p25'] * 100

Θα κατατάξω τα δεδομένα εισοδήματος σε bins όπως αυτά που φαίνονται στο διάγραμμα.

In [ ]:
bins = [0, 38, 41, 44, 48, 100]
labels = ['<38', '38 - 41', '41 - 44', '44 - 48', '>48']
zip_income['bins'] = pd.cut(zip_income['kfr_pooled_pooled_p25'], 
                            bins=bins, 
                            retbins=True, 
                            labels=labels)[0]

Στη συνέχεια ενώνω όλα τα δεδομένα σε ένα ενιαίο dataframe.

In [ ]:
zip_merged = pd.merge(zip_ec, zip_income, on='zip')

Ορίζω τα χρώματα, το μέγεθος και την χρωματική παλέτα του διαγράμματος.

In [ ]:
sb.set(style="darkgrid", rc={"figure.figsize":(9, 6)})
colorpalette = ['#cd0000', '#ffa500', '#ffe4c4', '#009acd', '#191970']
sb.set_palette(palette=colorpalette)

Φτιάχνω ένα scatter plot με τα δεδομένα. Στον άξονα $x$ έχω το μέσο εισόδημα ανά ZIP code, στον άξονα $y$ έχω τον δείκτη economic connectedness, ενώ τα διαφορετικά χρώματα διαχωρίζουν τα διαφορετικά επίπεδα upwards economic mobility, σύμφωνα με τα bins που έχουν οριστεί πιο πάνω. Τέλος, κάνω κάποιες αλλαγές στο legend ώστε να μπει στη σωστή σειρά και να έχει τη μορφή που θέλω και εμφανίζω το διάγραμμα.

In [ ]:
_ = sb.scatterplot(data=zip_merged,
                   x='med_inc_2018',
                   y='ec_zip',
                   hue='bins',
                   hue_order=labels,
                   alpha=0.7)
_.set(xlabel='Median Household Income in ZIP Codes (US$)',
      ylabel='Economic Connectedness',
      xlim=(20000, 100000))
_.set_xticks(range(30000, 110000, 10000))

#Change legend
handles, labels = plt.gca().get_legend_handles_labels()
order = [4, 3, 2, 1, 0]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], title='Upward Mobility', loc='lower right')
plt.show()

## Question 4

Για το 4ο ερώτημα θα φτιάξω το [Διάγραμμα 5α του 2ου paper](https://www.nature.com/articles/s41586-022-04997-3/figures/5). Αυτή τη φορά θα χρησιμοποιήσω την βιβλιοθήκη plotnine. Για τη δημιουργία των ξεχωριστών labels θα πρέπει να εγκαταστήσω το πακέτο adjustText.

In [ ]:
from plotnine import *
import adjustText

Αποθηκεύω στο dataframe schools_ec τα στοιχεία των σχολείων, τον δείκτη economic connectedness για τους γονείς των παιδιών που φοιτούν στα σχολεία και τον δείκτη friending bias των μαθητών που ανοίκουν στο κάτω μισό της εισοδηματικής κλίμακας. Για να βρούμε το ποσοστό των γονιών που βρίσκονται στο άνω μισό της εισοδηματικής κλίμακας θα διαιρέσουμε τον δείκτη mean exposure to high parental SES διά 2. Τέλος, μετατρέπουμε τις τιμές σε ποσοστά.

In [ ]:
schools_ec = pd.read_csv('social_capital_high_school.csv', 
                         dtype={'high_school': str}, 
                         usecols=['high_school', 
                                  'high_school_name', 
                                  'ec_parent_ses_hs', 
                                  'bias_parent_ses_hs', 
                                  'exposure_parent_ses_hs'])
schools_ec['high_parent_share'] = schools_ec['exposure_parent_ses_hs'] / 2 * 100
schools_ec['bias_parent_ses_hs'] = schools_ec['bias_parent_ses_hs'].apply(lambda x: x * 100)

Βρίσκουμε στο schools_ec τα συγκεκριμένα σχολεία που θέλουμε να ξεχωρίσουμε.

In [ ]:
schools_an = schools_ec[(schools_ec['high_school'] == '00941729') |
                        (schools_ec['high_school'] == '060474000432') |
                        (schools_ec['high_school'] == '170993000942') |
                        (schools_ec['high_school'] == '170993001185') |
                        (schools_ec['high_school'] == '170993003989') |
                        (schools_ec['high_school'] == '171449001804') |
                        (schools_ec['high_school'] == '250327000436') |
                        (schools_ec['high_school'] == '360009101928') |
                        (schools_ec['high_school'] == '370297001285') |
                        (schools_ec['high_school'] == '483702004138') |
                        (schools_ec['high_school'] == '250843001336') |
                        (schools_ec['high_school'] == '062271003230') |
                        (schools_ec['high_school'] == '010237000962') |
                        (schools_ec['high_school'] == '00846981') |
                        (schools_ec['high_school'] == '00852124')]

Φτιάχνω ένα λεξικό για να ορίσω το style των labels που θέλω να έχει το διάγραμμά μου.

In [ ]:
adjust_text_dict = {
    'expand_points': (2, 1.2),
    'color': 'turquoise',
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'turquoise',
        'lw': 1
    }
}

Τέλος, φτιάχνω το διάγραμμά μου, όπου στον άξονα $x$ έχουμε το ποσοστό γονέων με υψηλό εισόδημα και στον άξονα $y$ το ποσοτό friending bias γονέων με χαμηλό εισόδημα.

In [ ]:
g = ggplot(data=schools_ec) +\
    geom_point(mapping=aes(x='high_parent_share', y='bias_parent_ses_hs'), 
               alpha=0.2) +\
    geom_point(data=schools_an, 
               mapping=aes(x='high_parent_share', y='bias_parent_ses_hs'), 
               colour='turquoise') +\
    scale_y_reverse() +\
    labs(x='Share of high-parental-SES students (%)', 
         y='Friending bias among low-parental-SES students(%)') +\
    geom_label(mapping=aes(label='high_school_name', 
                           x='high_parent_share', 
                           y='bias_parent_ses_hs'),
               adjust_text=adjust_text_dict,
               alpha=0.8,
               data=schools_an) +\
    theme(figure_size=(12, 9))
print(g)

## Question 5

Για το 5ο ερώτημα θα δημιουργήσω το [3ο Διάγραμμα του 2ου paper](https://www.nature.com/articles/s41586-022-04997-3/figures/9). Αρχικά διαβάζω τα δεδομένα που σχετίζονται με το friendship bias για κάθε κολλέγιο και για κάθε γειτονιά.

In [ ]:
#College
college_bias = pd.read_csv('social_capital_college.csv', 
                           usecols=['college', 
                                    'mean_students_per_cohort', 
                                    'bias_own_ses_college'])
#ZIP Codes
zip_bias = pd.read_csv('social_capital_zip.csv', 
                       usecols=['zip', 
                                'nbhd_bias_zip', 
                                'num_below_p50'])

Στη συνέχεια διαβάζω τα δεδομένα που σχετίζονται με τις εθνικότητες στα δύο group. Για τον υπολογισμό των λευκών φοιτητών στα κολλέγια αφαιρώ από τους συνολικούς φοιτητές τα μερήδια των υπόλοιπων εθνικοτήτων.

In [ ]:
#College
college_race = pd.read_stata('college_characteristics.dta', 
                             columns=['college', 
                                      'zip', 
                                      'asian_or_pacific_share_fall_2000', 
                                      'black_share_fall_2000', 
                                      'hisp_share_fall_2000'])
college_race['share_white'] = 1 - college_race['asian_or_pacific_share_fall_2000'] - college_race['black_share_fall_2000'] - college_race['hisp_share_fall_2000']
#ZIP Codes
zip_race = pd.read_stata('zip_covariates.dta', 
                         columns=['zip', 
                                  'share_white_2018', 
                                  'share_black_2018', 
                                  'share_natam_2018', 
                                  'share_asian_2018', 
                                  'share_hawaii_2018', 
                                  'share_hispanic_2018'])

Υπολογίζω τον δείκτη diversity σύμφωνα με τον δείκτη [Herfindahl-Hirschman Index (HHI)](https://en.wikipedia.org/wiki/Herfindahl%E2%80%93Hirschman_index).

In [ ]:
#College
college_race = college_race.assign(diversity=lambda x: 1 
                                                       - ((x['asian_or_pacific_share_fall_2000']) ** 2 + 
                                                          (x['black_share_fall_2000']) ** 2 + 
                                                          (x['hisp_share_fall_2000']) ** 2 + 
                                                          (x['share_white']) ** 2))
#ZIP Codes
zip_race = zip_race.assign(diversity=lambda x: 1 
                                               - ((x['share_asian_2018']) ** 2 + 
                                                  (x['share_black_2018']) ** 2 + 
                                                  (x['share_hispanic_2018']) ** 2 + 
                                                  (x['share_white_2018']) ** 2 + 
                                                  (x['share_natam_2018']) ** 2 + 
                                                  (x['share_hawaii_2018']) ** 2))

Αφού έχω υπολογίση την μετρική diversity, φτιάχνω 20 bins από το 0 έως το 0.7 και κατηγοριοποιώ τις σειρές των Dataframes σύμφωνα με αυτά.

In [ ]:
bins = np.linspace(0., 0.7, 20)
zip_race['bins'] = pd.cut(zip_race['diversity'], 
                          bins=bins, 
                          labels=bins[1:])
college_race['bins'] = pd.cut(college_race['diversity'], 
                              bins=bins, 
                              labels=bins[1:])

Στη συνέχεια ενώνω τα dataframes που περιέχουν δεδομένα diversity με τα dataframes που περιέχουν δεδομένα για το friendship bias και ξεφορτώνομαι στήλες που δεν μου χρειάζονται πια. 

Μετά το merge φτιάχνω για τα δεδομένα κολλεγίων και τα δεδομένα γειτονιάς αντίστοιχα μία νέα στήλη (values) όπου θα χρησιμοποιηθεί μετά για τη δημιουργία του άξονα $x$ του διαγράμματος.

In [ ]:
#College
college_data = pd.merge(college_race, college_bias, on='college').drop(columns=['asian_or_pacific_share_fall_2000', 'black_share_fall_2000', 'hisp_share_fall_2000', 'share_white'])
college_data['value'] = college_data['mean_students_per_cohort'] * college_data['diversity']
#ZIP Codes
zip_data = pd.merge(zip_race, zip_bias, on='zip').drop(columns=['share_white_2018', 'share_black_2018', 'share_natam_2018', 'share_asian_2018', 'share_hawaii_2018', 'share_hispanic_2018'])
zip_data['value'] = zip_data['num_below_p50'] * zip_data['diversity']

Φτιάχνοντας το άξονα $x$, βρίσκουμε τον weighted μέσο όρο, όπως έχει οριστεί αυτός από την εκφώνηση. Για κάθε bin του δείκτη diversity αθροίζω όλες τις τιμές diversity που αντιστοιχούν στο συγκεκτιμένο bin, οι οποίες είναι πολλαπλασιασμένες με το weight που έχω ορίσει, και στο τέλος τις διαιρώ με το άθροισμα των weights για κάθε bin. Αποθηκεύω τα αποτελέσματα των πράξεων σε χωριστά dataframes, τα οποία θα έχουν μήκος μόλις 20 σειρές και προσθέτων μία στήλη (is) η οποία δείχνει κάθε φορά αν τα δεδομένα μας είναι δεδομένα κολλεγίου ή γειτονιάς. Μπορεί στο συγκεκριμένο βήμα η στήλη αυτή να φαίνεται περιττή, όμως θα χρησιμεύσει στην ενωποιήση όλων των δεδομένων και τον διαχωρισμό τους κατά τη δημουργεία του διαγράμματος.

In [ ]:
#College
x_college = college_data.groupby('bins').agg(sum=('value', 'sum'),
                                             sump=('mean_students_per_cohort', 'sum'))
x_college['x'] = x_college['sum'] / x_college['sump']
x_college['is'] = 'College'
#ZIP Codes
x_zip= zip_data.groupby('bins').agg(sum=('value', 'sum'),
                                    sump=('num_below_p50', 'sum'))
x_zip['x'] = x_zip['sum'] / x_zip['sump']
x_zip['is'] = 'Neighbourhood'

Ο άξονας $y$ φτιάχνεται πολύ πιο απλά από τον $x$. Έχωντας ήδη κατατάξει τα δεδομένα μας σε bins, η εύρεση του μέσου όρου για κάθε bin είναι απλή.

In [ ]:
#College
y_college = college_data.groupby('bins').agg(y=('bias_own_ses_college', 'mean'))
#ZIP Codes
y_zip = zip_data.groupby('bins').agg(y=('nbhd_bias_zip', 'mean'))

Σαν τελευταίο βήμα πριν τη δημιουργία του διαγράμματος ενώνω όλα μου τα δεδομένα σε ένα ενιαίο dataframe (all_data) και πολλαπλασιάζω τις τιμές του άξονα $y$ με το 100 ώστε να μετατραπούν σε ποσοστά.

In [ ]:
all_data = pd.DataFrame(data={'x': pd.concat([x_college['x'], x_zip['x']]),
                              'y': pd.concat([y_college['y'], y_zip['y']]),
                              'is': pd.concat([x_college['is'], x_zip['is']])
                              })
all_data['y'] = all_data['y'] * 100
all_data

Με τη χρήση της βιβλιοθήκης seaborn φτιάχνω το διάγραμμά μου.

In [ ]:
sb.set(style="darkgrid")
sb.set_palette(palette='tab10', n_colors=2, color_codes=['b', 'o'])
_ = sb.lmplot(data=all_data,
              x='x',
              y='y',
              hue='is',
              markers=['D', 'o'],
              legend=False)
_.set(xlabel='Racial Diversity (Herfindahl-Hirschman Index) in Group',
      ylabel='Friending Bias among Low-SES Individuals (%)',
      yticks=range(0, 25, 5))
_.fig.set_figwidth(7)
_.fig.set_figheight(4)
plt.legend(loc='upper left', title='')
plt.show()